# Tutorial 3: Fine-Tuning a Cohere LLM with Medical Data

• Find the dataset preparation  [Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2010%20-%20Create_Dataset_For_Cohere_Fine_Tuning.ipynb)  and the fine-tuning  [Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2010%20-%20Fine_Tuning_using_Cohere_for_Medical_Data.ipynb)  for this section at  [towardsai.net/book](http://towardsai.net/book).

Using a proprietary model simplifies the fine-tuning process by simply supplying sample inputs and outputs, with the platform managing the actual fine-tuning. For example, in a classification model, a typical input would be a pair of <text, label>.

Cohere offers a range of specialized models for specific use cases and different functions, including  [rerank](https://txt.cohere.com/rerank/),  [embedding](https://docs.cohere.com/docs/multilingual-language-models), and  [chat](https://cohere.com/chat), all accessible via APIs. Users can create  [custom models](https://txt.cohere.com/custom-command-models/)  for three primary objectives: 1) Generative tasks where the model produces text output, 2) Classification tasks where the model categorizes text, and 3) Rerank tasks to improve semantic search results.

In this tutorial, we are fine-tuning a proprietary LLM developed by  [Cohere](https://cohere.com/)  for medical text analysis to perform  [Named Entity Recognition (NER)](https://en.wikipedia.org/wiki/Named-entity_recognition). NER enables models to recognize multiple entities in text, such as names, locations, and dates. We will fine-tune a model to extract information about diseases, substances, and their interactions from medical paper abstracts.

## Cohere API

The Cohere platform provides a selection of base models designed for different purposes. You can choose between base models with quicker performance or command models with more advanced capabilities for generative tasks. Each type also has a “light” version for additional flexibility.

[Create an account](https://dashboard.cohere.com/welcome/register)  on their platform to use the Cohere API at dashboard.cohere.com. Navigate to the “API Keys” section to obtain a Trial key, which allows free usage with certain rate limitations. This key is not for production environments but offers an excellent opportunity to experiment with the models before using them for production.

Install the Cohere Python SDK to access their API:

pip install cohere

Build a Cohere object with your API key and a prompt to generate a response to your request. You can use the code below but change the API placeholder with your key:

    pip install cohere

Build a Cohere object with your API key and a prompt to generate a response to your request. You can use the code below but change the API placeholder with your key:

In [ ]:
import json
from fine_tuning_custom_utils.helper import get_cohere_api_key
COHERE_API_KEY = get_cohere_api_key()

In [ ]:
import cohere  

co = cohere.Client("<API_KEY>")

prompt = """The following article contains technical terms including diseases, drugs and chemicals. Create a list only of the diseases mentioned.

Progressive neurodegeneration of the optic nerve and the loss of retinal ganglion cells is a hallmark of glaucoma, the leading cause of irreversible blindness worldwide, with primary open-angle glaucoma (POAG) being the most frequent form of glaucoma in the Western world. While some genetic mutations have been identified for some glaucomas, those associated with POAG are limited and for most POAG patients, the etiology is still unclear. Unfortunately, treatment of this neurodegenerative disease and other retinal degenerative diseases is lacking. For POAG, most of the treatments focus on reducing aqueous humor formation, enhancing uveoscleral or conventional outflow, or lowering intraocular pressure through surgical means. These efforts, in some cases, do not always lead to a prevention of vision loss and therefore other strategies are needed to reduce or reverse the progressive neurodegeneration. In this review, we will highlight some of the ocular pharmacological approaches that are being tested to reduce neurodegeneration and provide some form of neuroprotection.

List of extracted diseases:"""

response = co.generate(  
    model='command',  
    prompt = prompt,  
    max_tokens=200,  
    temperature=0.750)

base_model = response.generations[0].text

print(base_model)

    - glaucoma  
    - primary open-angle glaucoma

The code uses the `cohere.Client()` method to input your API key. The above code also defines the prompt variable, which will contain instructions for the model.

The model’s objective for this experiment is to analyze a scientific paper’s abstract from the [PubMed website](https://pubmed.ncbi.nlm.nih.gov/) and identify a list of diseases. The cohere object’s `.generate()` method specifies the model type and provides the prompts and control parameters to achieve this.

The `max_tokens` parameter sets the limit for the number of new tokens the model can generate, and the temperature parameter controls the randomness level in the output.

The command model can identify diseases without examples or supplementary information.


## The Dataset

We will use the  [BC5CDR](https://paperswithcode.com/dataset/bc5cdr)  or BioCreative V Chemical Disease Relation Data. It comprises 1,500 manually annotated PubMed research papers, providing structured information on chemical-disease relations. The dataset is divided into training, validation, and testing sets containing 500 papers.

With this experiment, we aim to fine-tune a model that can identify and extract names of diseases/chemicals and their relationships from text. While research papers often describe relationships between chemicals and diseases in their abstracts, this information is typically unstructured. Manually finding “all chemicals influencing disease X” would require reading all papers mentioning “disease X.” Accurately extracting this structured information from unstructured text would facilitate more efficient searches.

Preprocess the dataset to adapt it for the Cohere service. It handles three file formats: CSV, JSONL, and plain text. We will use the JSONL format, which is consistent with the template below:

In [ ]:
{"prompt": "This is the first prompt",
"completion": "This is the first completion"}
{"prompt": "This is the second prompt",
"completion": "This is the second completion"}

💡The code is an example showing the extraction of disease names. Our final dataset will contain diseases, chemicals, and their corresponding relationships. We present a single step for extracting disease names to minimize the repetition of code. Please refer to [the notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2010%20-%20Create_Dataset_For_Cohere_Fine_Tuning.ipynb) at [towardsai.net/book](http://towardsai.net/book) for the complete preprocessing procedure and the resulting dataset.

Download the dataset in [JSON format](https://github.com/towardsai/rag-ebook-files/blob/main/bc5cdr.json) from [towardsai.net/book](http://towardsai.net/book) and open the JSON file using the code below. We also display a single row (passage) from the dataset to better illustrate the content and help understand the process. Each entry includes a text (which may be either a title or an abstract) and a list of entities that can be classified as either chemicals or diseases. For instance, in the example provided below, the first entity, Naloxone, is recognized as a chemical. The subsequent code will focus only on the information from the abstract, as the titles are short and provide limited details. (The printed output is simplified to improve understanding of the dataset and exclude non-essential information.)

In [ ]:
with open('bc5cdr.json') as json_file:
    data = json.load(json_file)

print(data[0])

    {'passages':  
    [{'document_id': '227508',  
    '**type**': 'title',  
    '**text**': 'Naloxone reverses the antihypertensive effect of clonidine.',  
    '**entities**': [  
    {'id': '0', 'text': ['Naloxone'], 'type': 'Chemical',},  
    {'id': '1', 'text': ['clonidine'], 'type': 'Chemical'}],  
    'relations': [...]},  
    {'document_id': '227508',  
    '**type**': 'abstract',  
    '**text**': 'In unanesthetized, spontaneously hypertensive rats the decrease in blood pressure and heart rate produced by intravenous clonidine, 5 to 20 micrograms/kg, was inhibited or reversed by nalozone, 0.2 to 2 mg/kg. The hypotensive effect of 100 mg/kg alpha-methyldopa was also partially reversed by naloxone. Naloxone alone did not affect either blood pressure or heart rate. In brain membranes from spontaneously hypertensive rats clonidine, 10(-8) to 10(-5) M, did not influence stereoselective binding of [3H]-naloxone (8 nM), and naloxone, 10(-8) to 10(-4) M, did not influence clonidine-suppressible binding of [3H]-dihydroergocryptine (1 nM). These findings indicate that in spontaneously hypertensive rats the effects of central alpha-adrenoceptor stimulation involve activation of opiate receptors. As naloxone and clonidine do not appear to interact with the same receptor site, the observed functional antagonism suggests the release of an endogenous opiate by clonidine or alpha-methyldopa and the possible role of the opiate in the central control of sympathetic tone.',  
    '**entities**': [  
    {'id': '2', 'text': ['hypertensive'], 'type': 'Disease'},  
    {'id': '3', 'text': ['clonidine'], 'type': 'Chemical'},  
    {'id': '4', 'text': ['nalozone'], 'type': 'Chemical'},  
    {'id': '5', 'text': ['hypotensive'], 'type': 'Disease'},  
    {'id': '6', 'text': ['alpha-methyldopa'], 'type': 'Chemical'},  
    {'id': '7', 'text': ['naloxone'], 'type': 'Chemical'},  
    {'id': '8', 'text': ['Naloxone'], 'type': 'Chemical'},  
    {'id': '9', 'text': ['hypertensive'], 'type': 'Disease'},  
    {'id': '10', 'text': ['clonidine'], 'type': 'Chemical'},  
    {'id': '11', 'text': ['[3H]-naloxone'], 'type': 'Chemical'},  
    {'id': '12', 'text': ['naloxone'], 'type': 'Chemical'},  
    {'id': '13', 'text': ['clonidine'], 'type': 'Chemical'},  
    {'id': '14', 'text': ['[3H]-dihydroergocryptine'], 'type': 'Chemical'},  
    {'id': '15', 'text': ['hypertensive'], 'type': 'Disease'},  
    {'id': '16', 'text': ['naloxone'], 'type': 'Chemical',},  
    {'id': '17', 'text': ['clonidine'], 'type': 'Chemical'},  
    {'id': '18', 'text': ['clonidine'], 'type': 'Chemical'},  
    {'id': '19', 'text': ['alpha-methyldopa'], 'type': 'Chemical'}],  
    'relations': [...]}],  
    'dataset_type': 'train'}


We can loop through the dataset, extracting abstracts and related entities while including training instructions. There are two sets of instructions: the first helps the model understand the job, and the second shows how to construct the response:

In [ ]:
instruction = "The following article contains technical terms including diseases, drugs and chemicals. Create a list only of the diseases mentioned.\n\n"
output_instruction = "\n\nList of extracted diseases:\n"

The `instruction` variable sets the rules, and the `output_instruction` specifies the intended format for the output. Now, cycle through the dataset and format each instance:

In [ ]:
the_list = []
for item in data:
  chems = []
  dis = []

  if item['dataset_type'] == "test": continue;

  for ent in item['passages'][1]['entities']:
    if ent['type'] == "Chemical":
      if ent['text'][0] not in chems:
        chems.append( ent['text'][0] )

    if ent['type'] == "Disease":
      if ent['text'][0] not in dis:
        dis.append( ent['text'][0] )

Preparing each sample from the dataset requires iterating through all annotations and selecting only those related to diseases. This is necessary because the dataset also contains additional chemical labels. This will result in a dictionary with two keys: `prompt` and `completion`. The prompt key will consist of the paper abstract combined with specific instructions, and the `completion` key will list each disease name on a separate line.

This code will convert and save the dataset in JSONL format:

In [ ]:
# Writing to sample.json
with open("disease_instruct_all.jsonl", "w") as outfile:
  for item in the_list:
    outfile.write(json.dumps(item) + "\n")

The processed dataset is saved in a file named `disease_instruct_all.jsonl`. This file combines the training and validation sets to create 1,000 samples. The complete dataset comprises 3,000 samples, divided into three categories: 1,000 for diseases, 1,000 for chemicals, and 1,000 for their relationships.

💡The link to the final [preprocessed dataset](https://github.com/towardsai/rag-ebook-files/blob/main/both_rel_instruct_all.jsonl) is accessible at [towardsai.net/book](http://towardsai.net/book).

## Fine-Tuning

The Cohere platform offers advanced options for extending the training duration or adjusting the learning rate. Refer to their guide on  [Training Custom Models](https://docs.cohere.com/docs/finetuning)  for a comprehensive understanding.

Navigate to the models’ page on the sidebar and click the “Create a custom model” button. You will be prompted to select the model type; for this example, choose the Generate option.

Next, upload the dataset from the previous step or a custom dataset. Click the “Review data” button to preview a few samples from the dataset. This ensures that the platform correctly interprets your data. If everything looks correct, proceed by clicking the “Continue” button.

Next, choose a nickname for your model. You can also modify training hyperparameters by clicking the “HYPERPARAMETERS (OPTIONAL)” link. Options include `train_steps` for training duration, `learning_rate` for adjusting how quickly the model adapts, and `batch_size` for the number of samples processed in each iteration. While the default parameters are generally effective, you can experiment with this.

Once you’re ready, click “Initiate training.” Cohere will email you to notify you that the fine-tuning process is complete and provide you with the model ID for use in your API.

### Extract Disease Names

Use the previously created prompt, but with the model ID of the network, we just fine-tuned:

In [ ]:
response = co.generate(  
    model='2075d3bc-eacf-472e-bd26-23d0284ec536-ft',  
    prompt=prompt,  
    max_tokens=200,  
    temperature=0.750)

disease_model = response.generations[0].text

print(disease_model)

The results show that the model can now recognize a wide range of diseases, demonstrating the effectiveness of the fine-tuning method.

### Extract Chemical Names

We also compared the performance of the baseline model with the fine-tuned model in extracting chemical names. We will only show each model’s prompt and output to avoid unnecessary code mentions. We used the following prompt to extract information from a text in the test set:

In [ ]:
prompt = """The following article contains technical terms including diseases, drugs and chemicals. Create a list only of the chemicals mentioned.

To test the validity of the hypothesis that hypomethylation of DNA plays an important role in the initiation of carcinogenic process, 5-azacytidine (5-AzC) (10 mg/kg), an inhibitor of DNA methylation, was given to rats during the phase of repair synthesis induced by the three carcinogens, benzo[a]-pyrene (200 mg/kg), N-methyl-N-nitrosourea (60 mg/kg) and 1,2-dimethylhydrazine (1,2-DMH) (100 mg/kg). The initiated hepatocytes in the liver were assayed as the gamma-glutamyltransferase (gamma-GT) positive foci formed following a 2-week selection regimen consisting of dietary 0.02% 2-acetylaminofluorene coupled with a necrogenic dose of CCl4. The results obtained indicate that with all three carcinogens, administration of 5-AzC during repair synthesis increased the incidence of initiated hepatocytes, for example 10-20 foci/cm2 in 5-AzC and carcinogen-treated rats compared with 3-5 foci/cm2 in rats treated with carcinogen only. Administration of [3H]-5-azadeoxycytidine during the repair synthesis induced by 1,2-DMH further showed that 0.019 mol % of cytosine residues in DNA were substituted by the analogue, indicating that incorporation of 5-AzC occurs during repair synthesis. In the absence of the carcinogen, 5-AzC given after a two thirds partial hepatectomy, when its incorporation should be maximum, failed to induce any gamma-GT positive foci. The results suggest that hypomethylation of DNA per se may not be sufficient for initiation. Perhaps two events might be necessary for initiation, the first caused by the carcinogen and a second involving hypomethylation of DNA.

List of extracted chemicals:"""


The output of the base model:

    - 5-azacytidine (5-AzC)  
    - benzo[a]-pyrene  
    - N-methyl-N-nitrosourea  
    - 1,2-dimethylhydrazine  
    - CCl4  
    - 2-acetylaminofluorene

The output of the custom fine-tuned model:

    - 5-azacytidine  
    - 5-AzC  
    - benzo[a]-pyrene  
    - N-methyl-N-nitrosourea  
    - 1,2-dimethylhydrazine  
    - 1,2-DMH  
    - 2-acetylaminofluorene  
    - CCl4  
    - [3H]-5-azadeoxycytidine  
    - cytosine

The custom model is better for our specific task and adapts readily based on the samples.

### Extract Relations

Here, the model will extract relationships between chemicals and the diseases they affect. It is a complex task that may present difficulties for the base model.

Introduce the prompt from the test set:

In [ ]:
prompt = """The following article contains technical terms including diseases, drugs and chemicals. Create a list only of the influences between the chemicals and diseases mentioned.

The yield of severe cirrhosis of the liver (defined as a shrunken finely nodular liver with micronodular histology, ascites greater than 30 ml, plasma albumin less than 2.2 g/dl, splenomegaly 2-3 times normal, and testicular atrophy approximately half normal weight) after 12 doses of carbon tetrachloride given intragastrically in the phenobarbitone-primed rat was increased from 25% to 56% by giving the initial "calibrating" dose of carbon tetrachloride at the peak of the phenobarbitone-induced enlargement of the liver. At this point it was assumed that the cytochrome P450/CCl4 toxic state was both maximal and stable. The optimal rat size to begin phenobarbitone was determined as 100 g, and this size as a group had a mean maximum relative liver weight increase 47% greater than normal rats of the same body weight. The optimal time for the initial dose of carbon tetrachloride was after 14 days on phenobarbitone.

List of extracted influences:"""

The output generated by the base model:

    severe cirrhosis of the liver influences shrinking, finely nodular, ascites, plasma albumin, splenomegaly, testicular atrophy, carbon tetrachloride, phenobarbitone

The output generated by the custom model:

    - Chemical phenobarbitone influences disease cirrhosis of the liver  
    - Chemical carbon tetrachloride influences disease cirrhosis of the liver

The base model tries to establish links within the text, but the custom fine-tuned model delivers well-formatted output, linking each chemical to the appropriate disease. This task is difficult for a general-purpose model, but it demonstrates fine-tuning efficiency with just a few thousand samples of the task.